In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.utils import to_categorical
# from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
#from kerastuner.tuners import RandomSearch

In [2]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

In [3]:
#preprocessing
X=train.drop(labels=['label'],axis=1)
y=train['label']

#shuffle training data
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
#one hot encode labels
Y=to_categorical(y,num_classes=10)
#normalize
X=X/255.0
test=test/255.0
#reshape image in 3 dimensions
X=X.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
model=keras.Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.5),
    Dense(10,activation='softmax')
])

In [5]:
#compiling model
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
#fitting model
# Define the callback
#sets a learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
# When fitting the model, add the callback
history = model.fit(X_train, Y_train, batch_size=100, epochs=15 , validation_data=(X_test, Y_test), callbacks=[reduce_lr])

Epoch 1/15
336/336 [==============================] - 5s 15ms/step - loss: 1.6044 - accuracy: 0.4636 - val_loss: 0.6167 - val_accuracy: 0.8193 - lr: 0.0010
Epoch 2/15
336/336 [==============================] - 5s 15ms/step - loss: 0.5942 - accuracy: 0.8100 - val_loss: 0.3634 - val_accuracy: 0.8877 - lr: 0.0010
Epoch 3/15
336/336 [==============================] - 5s 16ms/step - loss: 0.3975 - accuracy: 0.8762 - val_loss: 0.2416 - val_accuracy: 0.9287 - lr: 0.0010
Epoch 4/15
336/336 [==============================] - 6s 17ms/step - loss: 0.2862 - accuracy: 0.9091 - val_loss: 0.1750 - val_accuracy: 0.9483 - lr: 0.0010
Epoch 5/15
336/336 [==============================] - 6s 17ms/step - loss: 0.2229 - accuracy: 0.9307 - val_loss: 0.1383 - val_accuracy: 0.9582 - lr: 0.0010
Epoch 6/15
336/336 [==============================] - 6s 17ms/step - loss: 0.1861 - accuracy: 0.9407 - val_loss: 0.1263 - val_accuracy: 0.9593 - lr: 0.0010
Epoch 7/15
336/336 [==============================] - 6s 17ms/st

In [6]:
results=model.predict(test)
results=np.argmax(results,axis=1)
results=pd.Series(results,name="label")
submission=pd.concat([pd.Series(range(1,len(results)+1),name="ImageId"),results],axis=1)
submission.to_csv('data/submission.csv',index=False)

875/875 [==============================] - 2s 2ms/step
